In [1]:
#Import all the important files 
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import date
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder

In [2]:
#creating a data frame from CSV
customer = pd.read_csv("Customer.csv")
prod_cat_info = pd.read_csv("prod_cat_info.csv")
transactions=pd.read_csv("Transactions.csv")


In [3]:
customer.shape

(5647, 4)

In [4]:
prod_cat_info.shape

(23, 4)

In [5]:
transactions.shape

(23053, 10)

In [6]:
# changing column label to similar in all tables
customer.rename(columns = {'customer_Id':'cust_id'}, inplace = True)


In [7]:
# removing row if customer_id/prod_sub_cat_code is null from customer and prod_cat_info dataframe
customer.dropna(subset=['cust_id'], inplace=True)
prod_cat_info.dropna(subset=['prod_sub_cat_code'],inplace=True)
customer['Gender'].fillna('F', inplace=True)

# Convert 'DOB' column to datetime with the correct format
customer['DOB'] = pd.to_datetime(customer['DOB'], format='%d-%m-%Y')

# Fill missing values in 'DOB' with a default date
customer['DOB'].fillna(pd.to_datetime('1900-01-01'), inplace=True)

# Rest of your code
customer['city_code'] =abs (pd.to_numeric(customer['city_code'], errors='coerce').fillna(-1).astype(int))

# Verify the data types
print(customer.dtypes)




cust_id               int64
DOB          datetime64[ns]
Gender               object
city_code             int64
dtype: object


In [8]:
# creating super table by joining customer and prod_cat_info to transactions
df1=transactions.merge(customer,how='left',on='cust_id')
transaction_master_bi=pd.merge(left=df1, right=prod_cat_info,how='left',left_on=['prod_cat_code','prod_subcat_code'],right_on=['prod_cat_code','prod_sub_cat_code'])
transaction_master_bi.drop(columns='prod_sub_cat_code',axis=1,inplace=True)


In [9]:
# correceting data type of numeric and date columns
transaction_master_bi['tran_date'] = pd.to_datetime(transaction_master_bi['tran_date'],infer_datetime_format=True)
transaction_master_bi['Qty']=abs(transaction_master_bi['Qty'])
transaction_master_bi['Rate']=abs(transaction_master_bi['Rate'])
transaction_master_bi['total_amt']=abs(transaction_master_bi['total_amt'])

/var/folders/4x/bhr05fqs1xq2l403bnmgyvf00000gn/T/ipykernel_90521/2085579646.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  transaction_master_bi['tran_date'] = pd.to_datetime(transaction_master_bi['tran_date'],infer_datetime_format=True)
/var/folders/4x/bhr05fqs1xq2l403bnmgyvf00000gn/T/ipykernel_90521/2085579646.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  transaction_master_bi['tran_date'] = pd.to_datetime(transaction_master_bi['tran_date'],infer_datetime_format=True)


In [10]:
transaction_master_bi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23053 entries, 0 to 23052
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   transaction_id    23053 non-null  int64         
 1   cust_id           23053 non-null  int64         
 2   tran_date         23053 non-null  datetime64[ns]
 3   prod_subcat_code  23053 non-null  int64         
 4   prod_cat_code     23053 non-null  int64         
 5   Qty               23053 non-null  int64         
 6   Rate              23053 non-null  int64         
 7   Tax               23053 non-null  float64       
 8   total_amt         23053 non-null  float64       
 9   Store_type        23053 non-null  object        
 10  DOB               23053 non-null  datetime64[ns]
 11  Gender            23053 non-null  object        
 12  city_code         23053 non-null  int64         
 13  prod_cat          23053 non-null  object        
 14  prod_subcat       2305

In [11]:
# handling null values
transaction_master_bi.dropna(subset=['transaction_id','cust_id','prod_subcat_code','prod_cat_code','tran_date'],inplace=True)
transaction_master_bi['Store_type'].fillna('NA',inplace=True)
transaction_master_bi['Qty'].fillna(1,inplace=True)
transaction_master_bi['Qty'] = transaction_master_bi['Qty'].apply(lambda x: 1 if x == 0 else x)
transaction_master_bi['Rate'].fillna(transaction_master_bi['Rate'].mean(),inplace=True)

In [12]:
transaction_master_bi.sample()

,transaction_id,cust_id,tran_date,prod_subcat_code,prod_cat_code,Qty,Rate,Tax,total_amt,Store_type,DOB,Gender,city_code,prod_cat,prod_subcat
20540,75163718042,274186,2011-05-30,3,2,4,878,368.76,3880.76,MBR,1982-04-20,M,3,Footwear,Women


In [13]:
avg_tax=transaction_master_bi['Tax'].mean()
amount_before_tax=transaction_master_bi['Rate']*transaction_master_bi['Qty']
avg_amount=amount_before_tax.mean()
avg_tax_rate=avg_tax/avg_amount
avg_tax=avg_tax_rate*avg_amount
transaction_master_bi['Tax'].fillna(avg_tax,inplace=True)
total_amount=transaction_master_bi['Rate']*transaction_master_bi['Qty']+transaction_master_bi['Tax']
avg_total_amount=total_amount.mean()
transaction_master_bi['total_amt'].fillna(avg_total_amount,inplace=True)

In [14]:
ftd=transaction_master_bi.groupby('cust_id').min()['tran_date']
transaction_master_bi['First Purchase Date']=transaction_master_bi.apply(lambda row: ftd.loc[row['cust_id']], axis=1)

In [15]:
from datetime import date
today = date.today()

# Convert 'First Purchase Date' and 'DOB' to datetime format if not already
transaction_master_bi['First Purchase Date'] = pd.to_datetime(transaction_master_bi['First Purchase Date'])
transaction_master_bi['DOB'] = pd.to_datetime(transaction_master_bi['DOB'])

# Print data types of relevant columns
print(transaction_master_bi[['First Purchase Date', 'DOB']].dtypes)

# Calculate 'user_age_in_system_month'
transaction_master_bi['user_age_in_system_month'] = (today - transaction_master_bi['First Purchase Date'].dt.date) / np.timedelta64(1, 'm')

# Calculate 'user_age_in_year'
transaction_master_bi['user_age_in_year'] = (today - transaction_master_bi['DOB'].dt.date) / np.timedelta64(1, 'D')/365
transaction_master_bi['user_age_in_year'] = pd.to_numeric(transaction_master_bi['user_age_in_year'])

# Group by 'cust_id' and count the total transactions
tot_txn = transaction_master_bi.groupby('cust_id').count()['transaction_id']

# Create a new column 'lifetime_txn' using the total transactions
transaction_master_bi['lifetime_txn']=transaction_master_bi.apply(lambda row: tot_txn.loc[row['cust_id']], axis=1)
df=transaction_master_bi[['cust_id','prod_cat','Qty','total_amt']]



First Purchase Date    datetime64[ns]
DOB                    datetime64[ns]
dtype: object


In [16]:
df['avg_price']=df.apply(lambda row: row['total_amt'] / row['Qty'], axis=1)
df1=df.groupby(['cust_id','prod_cat']).mean()['avg_price'].to_frame().reset_index()
df1.rename(columns = {'avg_price':'user_aov'}, inplace = True)
transaction_master_bi=pd.merge(left=transaction_master_bi, right=df1,how='left',left_on=['cust_id','prod_cat'],right_on=['cust_id','prod_cat'])


/var/folders/4x/bhr05fqs1xq2l403bnmgyvf00000gn/T/ipykernel_90521/2398752847.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['avg_price']=df.apply(lambda row: row['total_amt'] / row['Qty'], axis=1)


In [17]:
age_ranges = pd.IntervalIndex.from_tuples([(0, 18), (19, 25), (26, 35), (36, 120)])
transaction_master_bi["age_range"] = pd.cut(transaction_master_bi["user_age_in_year"], age_ranges, labels=["<18", "18-25", "25-35", ">35"])

In [18]:
transaction_master_bi["product_code"]=transaction_master_bi.apply(lambda row: str(row['prod_cat']) +' '+ str(row['prod_subcat']), axis=1)

In [19]:
Product_purchase=transaction_master_bi.groupby(['prod_subcat','prod_cat']).agg({'transaction_id': 'nunique','cust_id': 'nunique','Qty':'sum','user_aov':'mean'}).reset_index()
Product_purchase.rename(columns = {'transaction_id':'unique_orders','cust_id':'users','Qty':'tot_orders','user_aov':'unit_price'},inplace=True)
Product_Master_bi=pd.merge(left=prod_cat_info, right=Product_purchase,how='left',left_on=['prod_cat','prod_subcat'],right_on=['prod_cat','prod_subcat'])
Product_Master_bi['product_code']=Product_Master_bi.apply(lambda row: str(row['prod_cat']) +' '+ str(row['prod_subcat']), axis=1)
platform_orders=sum(Product_Master_bi['tot_orders'])
Product_Master_bi['order_percentage']=Product_Master_bi['tot_orders']/platform_orders

In [20]:
customer_purchase=transaction_master_bi.groupby(['cust_id','prod_cat','prod_subcat']).agg({'transaction_id': 'nunique','Qty':'sum','total_amt':'mean','user_aov':'mean'}).reset_index()
customer_purchase.rename(columns = {'transaction_id':'unique_orders','Qty':'tot_orders','total_amt':'aov','user_aov':'unit_price'},inplace=True)
customer_purchase['product_code']=customer_purchase.apply(lambda row: str(row['prod_cat']) +' '+ str(row['prod_subcat']), axis=1)

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
Product_Master_bi['content'] = Product_Master_bi['product_code']
# Create TF-IDF matrix
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(Product_Master_bi['content'])

# Compute the cosine similarity
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
product_mapping = {}
for idx, row in Product_Master_bi.iterrows():
    product_mapping[(row['product_code'])] = idx

In [22]:
# Function to get recommendations
def get_similar_products(product_info, cosine_sim=cosine_sim):
    product_indices = [product_mapping.get((product_code), -1) for product_code in product_info]
    product_indices = [idx for idx in product_indices if idx != -1]  # Remove -1 (not found) from the list
    all_recommendations = []

    for product_index in product_indices:
        sim_scores = list(enumerate(cosine_sim[product_index]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[0:]  # Get the top 3 similar products (excluding itself)
        indices = [index for index, _ in sim_scores]
        recommendations = Product_Master_bi.iloc[indices].copy()
        recommendations['sim_score'] = [sim_score for _, sim_score in sim_scores]  # Add sim_score column
        all_recommendations.append(recommendations)
    final_df=pd.concat(all_recommendations, ignore_index=True)
    net_score=final_df.groupby('product_code').sum()['sim_score'].to_frame().reset_index()
    recom=pd.merge(left=final_df, right=net_score,how='left',left_on=['product_code'],right_on=['product_code'])
    recom.drop(columns='sim_score_x',inplace=True)
    recom.drop_duplicates()
    return recom

In [23]:
def user_recommendation(user_id,top_n=10):
    user_data=customer_purchase[customer_purchase['cust_id']==user_id]
    user_unit_price_avg=sum(user_data['aov']*user_data['unique_orders'])/sum(user_data['tot_orders'])
    user_products=list(user_data['product_code'])
    similar_products=get_similar_products(user_products, cosine_sim=cosine_sim)
    similar_products['product_score']=similar_products['order_percentage']+similar_products['sim_score_y']
    similar_products['product_score'] = similar_products.apply(lambda row: row['product_score'] + 0.1 if (row['unit_price'] >=0.9*user_unit_price_avg) & (row['unit_price']<=1.1*user_unit_price_avg) else row['product_score'], axis=1)
    sorted_similar_products = similar_products.sort_values(by='product_score',ascending=False)
    recommended_products=sorted_similar_products[['product_code','product_score']]
    return recommended_products.head(top_n)


In [24]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
transaction_master_bi['prod_code'] = label_encoder.fit_transform(transaction_master_bi['product_code'])
transaction_master_bi['sex'] = label_encoder.fit_transform(transaction_master_bi['Gender'])
transaction_master_bi['age_split'] = label_encoder.fit_transform(transaction_master_bi['age_range'])
transaction_master_bi['city'] = label_encoder.fit_transform(transaction_master_bi['city_code'])
edf=transaction_master_bi[['product_code','prod_code']].drop_duplicates()
# Create user-item matrix with additional features
user_item_matrix = pd.pivot_table(transaction_master_bi, values='Qty', index=['cust_id','sex', 'age_split', 'city'],columns='prod_code',aggfunc="sum", fill_value=0,)
user_item_matrix.fillna(0, inplace=True)  # Fill missing values

# Calculate cosine similarity between users
user_similarity = cosine_similarity(user_item_matrix)
cust_ids = user_item_matrix.index.get_level_values('cust_id').unique()
user_similarity_df = pd.DataFrame(user_similarity, index=cust_ids, columns=cust_ids)


# Function to get recommendations for a given user
def get_recommendations(user_id, user_similarity_matrix, user_item_matrix, top_n=10):
    similar_users = user_similarity_matrix[user_id]
    sm_df = similar_users.to_frame().sort_values(by=[user_id], ascending=False).reset_index()
    sm_df = sm_df[(sm_df[user_id] != 0) & (sm_df['cust_id'] != user_id)]

    product_scores = {}

    for other_user in sm_df['cust_id']:
        other_user_items = user_item_matrix.loc[other_user]
        relevant_items = other_user_items.columns[other_user_items.any()]

        for item in relevant_items:
            product_scores[item] = product_scores.get(item, 0) + other_user_items[item].sum() * sm_df.loc[sm_df['cust_id'] == other_user, [user_id]].values[0]

    user_items = user_item_matrix.loc[user_id]
    keys_to_remove = user_items.columns[user_items.any()]

    # Remove items already purchased by the user
    product_scores = {key: value for key, value in product_scores.items()}# if key not in keys_to_remove}

    # Create DataFrame from product_scores
    s = pd.DataFrame(list(product_scores.items()), columns=['prod_code', 'score'])
    s = s.sort_values(by='score', ascending=False).head(top_n)

    # Merge with edf to get additional product information
    recom = pd.merge(s, edf, on='prod_code', how='left').drop(columns='prod_code', axis=1)
    recom['score']=recom['score'].astype(int)
    return recom
user_id =274609
Recommendations=get_recommendations(user_id, user_similarity_df, user_item_matrix,top_n=10)
Recommendations['score']=Recommendations['score'].astype(int)
recom=Recommendations["product_code"]


In [25]:
recom

0            Electronics Mobiles
1                 Books Academic
2    Electronics Audio and video
3              Books Non-Fiction
4                  Clothing Kids
5                      Books DIY
6          Home and kitchen Bath
7                 Clothing Women
8                  Books Fiction
9                     Bags Women
Name: product_code, dtype: object

In [26]:
def hybrid_method_recom(user_id,top_n):
    product_recommendation_content=user_recommendation(user_id,top_n).reset_index()
    product_recommendation_collabarative=get_recommendations(user_id,user_similarity_df,user_item_matrix,top_n)
    product_recommendation_content.drop(columns='index',axis=1,inplace=True)
    product_recommendation_collabarative.rename(columns = {'score':'product_score'}, inplace = True)
    n1=sum(product_recommendation_collabarative['product_score'])
    n2=sum(product_recommendation_content['product_score'])
    product_recommendation_content['normalised_score']=product_recommendation_content['product_score']/n2
    product_recommendation_collabarative['normalised_score']=product_recommendation_collabarative['product_score']/n1
    combined = pd.merge(product_recommendation_collabarative, product_recommendation_content, on='product_code', how='outer')
    combined.fillna(0,inplace=True)
    combined.drop(columns=['product_score_y','product_score_x'],axis=1,inplace=True)
    combined.drop_duplicates(inplace=True)
    combined['final_score']=combined['normalised_score_x']*0.6+combined['normalised_score_y']*0.4
    all_recom=combined.sort_values(by='final_score',ascending=False)
    recommendations=all_recom.head(top_n)
    return recommendations

In [27]:
user_id=266783
top_n=15
hybrid_recom=hybrid_method_recom(user_id,top_n)
hybrid_recom

,product_code,normalised_score_x,normalised_score_y,final_score
0,Footwear Mens,0.238041,0.087021,0.177633
3,Clothing Mens,0.147102,0.087104,0.123103
6,Books Non-Fiction,0.095347,0.058901,0.080768
16,Books Fiction,0.042612,0.044229,0.043259
9,Home and kitchen Tools,0.046204,0.000000,0.027722
10,Books Children,0.046204,0.000000,0.027722
11,Footwear Women,0.045714,0.000000,0.027429
12,Home and kitchen Furnishing,0.045061,0.000000,0.027037
13,Clothing Kids,0.043592,0.000000,0.026155
14,Footwear Kids,0.043429,0.000000,0.026057


In [28]:
#accuracy check
from sklearn.model_selection import train_test_split
transaction_master_bi_train,transaction_master_bi_test=train_test_split(transaction_master_bi,test_size=0.25,random_state=42)

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
Product_purchase_train=transaction_master_bi_train.groupby(['prod_subcat','prod_cat']).agg({'transaction_id': 'nunique','cust_id': 'nunique','Qty':'sum','user_aov':'mean'}).reset_index()
Product_purchase_train.rename(columns = {'transaction_id':'unique_orders','cust_id':'users','Qty':'tot_orders','user_aov':'unit_price'},inplace=True)
Product_Master_bi_train=pd.merge(left=prod_cat_info, right=Product_purchase_train,how='left',left_on=['prod_cat','prod_subcat'],right_on=['prod_cat','prod_subcat'])
Product_Master_bi_train['product_code']=Product_Master_bi_train.apply(lambda row: str(row['prod_cat']) +' '+ str(row['prod_subcat']), axis=1)
platform_orders_train=sum(Product_Master_bi_train['tot_orders'])
Product_Master_bi_train['order_percentage']=Product_Master_bi_train['tot_orders']/platform_orders
Product_Master_bi_train['content'] = Product_Master_bi_train['product_code']
# Create TF-IDF matrix
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(Product_Master_bi['content'])

# Compute the cosine similarity
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
product_mapping = {}
for idx, row in Product_Master_bi_train.iterrows():
    product_mapping[(row['product_code'])] = idx

In [30]:
customer_purchase_train=transaction_master_bi_train.groupby(['cust_id','prod_cat','prod_subcat']).agg({'transaction_id': 'nunique','Qty':'sum','total_amt':'mean','user_aov':'mean'}).reset_index()
customer_purchase_train.rename(columns = {'transaction_id':'unique_orders','Qty':'tot_orders','total_amt':'aov','user_aov':'unit_price'},inplace=True)
customer_purchase_train['product_code']=customer_purchase_train.apply(lambda row: str(row['prod_cat']) +' '+ str(row['prod_subcat']), axis=1)

In [31]:
def get_similar_products_accuracy(product_info, cosine_sim=cosine_sim):
    product_indices = [product_mapping.get((product_code), -1) for product_code in product_info]
    product_indices = [idx for idx in product_indices if idx != -1]  # Remove -1 (not found) from the list
    all_recommendations = []

    for product_index in product_indices:
        sim_scores = list(enumerate(cosine_sim[product_index]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[0:]
        indices = [index for index, _ in sim_scores]
        recommendations = Product_Master_bi_train.iloc[indices].copy()
        recommendations['sim_score'] = [sim_score for _, sim_score in sim_scores]  # Add sim_score column
        all_recommendations.append(recommendations)
    final_df=pd.concat(all_recommendations, ignore_index=True)
    net_score=final_df.groupby('product_code').sum()['sim_score'].to_frame().reset_index()
    recom=pd.merge(left=final_df, right=net_score,how='left',left_on=['product_code'],right_on=['product_code'])
    recom.drop(columns='sim_score_x',inplace=True)
    recom.drop_duplicates(inplace=True)
    return recom

In [32]:
def user_recommendation_accuracy(user_id,top_n=10):
    user_data=customer_purchase_train[customer_purchase_train['cust_id']==user_id]
    user_unit_price_avg=sum(user_data['aov']*user_data['unique_orders'])/sum(user_data['tot_orders'])
    user_products=list(user_data['product_code'].unique())
    similar_products=get_similar_products_accuracy(user_products, cosine_sim=cosine_sim)
    similar_products['product_score']=similar_products['order_percentage']+similar_products['sim_score_y']
    similar_products['net_score'] = similar_products.apply(lambda row: row['product_score'] + 0.3 if (row['unit_price'] >=0.7*user_unit_price_avg) & (row['unit_price']<=1.3*user_unit_price_avg) else row['product_score'], axis=1)
    sorted_similar_products = similar_products.sort_values(by='net_score',ascending=False)
    recommended_products=sorted_similar_products[['product_code','product_score']]
    return recommended_products.head(top_n)

In [33]:
def hybrid_method_recom_accuracy(user_id,top_n):
    product_recommendation_content=user_recommendation_accuracy(user_id,top_n).reset_index()
    product_recommendation_collabarative=get_recommendations(user_id,user_similarity_df_train,user_item_matrix_train,top_n)
    product_recommendation_content.drop(columns='index',axis=1,inplace=True)
    product_recommendation_collabarative.rename(columns = {'score':'product_score'}, inplace = True)
    print(product_recommendation_content.columns)
    n1=sum(product_recommendation_collabarative['product_score'])
    n2=sum(product_recommendation_content['product_score'])
    product_recommendation_content['normalised_score']=product_recommendation_content['product_score']/n2
    product_recommendation_collabarative['normalised_score']=product_recommendation_collabarative['product_score']/n1
    combined = pd.merge(product_recommendation_collabarative, product_recommendation_content, on='product_code', how='outer')
    combined.fillna(0,inplace=True)
    combined.drop(columns=['product_score_y','product_score_x'],axis=1,inplace=True)
    combined.drop_duplicates(inplace=True)
    combined['final_score']=combined['normalised_score_x']*0.5+combined['normalised_score_y']*0.5
    all_recom=combined.sort_values(by='final_score',ascending=False)
    recommendations=all_recom.head(top_n)
    return recommendations



In [34]:
from sklearn.metrics import precision_score, recall_score, f1_score, average_precision_score
precision_scores = []
recall_scores = []
f1_scores = []
average_precision_scores = []
for user_id in cust_ids_to_evaluate:
    Recommendations =hybrid_method_recom_accuracy(user_id,top_n=10)
    actual_purchases_for_user =list(set(actual_test_set.get(user_id, [])))
    predicted_purchases_for_user = Recommendations['product_code'].tolist()

    # True labels for the user
    true_labels = [1 if prod_code in actual_purchases_for_user else 0 for prod_code in actual_test_set[user_id]]

    # Predicted scores for the recommendations
    predicted_scores = [1 if prod_code in predicted_purchases_for_user else 0 for prod_code in actual_test_set[user_id]]

    # Calculate precision, recall, F1-score, and average precision
    precision = precision_score(true_labels, predicted_scores)
    recall = recall_score(true_labels, predicted_scores)
    f1 = f1_score(true_labels, predicted_scores)
    average_precision = average_precision_score(true_labels, predicted_scores)

    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)
    average_precision_scores.append(average_precision)

# Calculate the mean scores across users
mean_precision = sum(precision_scores) / len(precision_scores)
mean_recall = sum(recall_scores) / len(recall_scores)
mean_f1 = sum(f1_scores) / len(f1_scores)
mean_average_precision = sum(average_precision_scores) / len(average_precision_scores)
print(f"Mean Precision: {mean_precision:.4f}")
print(f"Mean Recall: {mean_recall:.4f}")
print(f"Mean F1-Score: {mean_f1:.4f}")
print(f"Mean Average Precision: {mean_average_precision:.4f}")

NameError: name 'cust_ids_to_evaluate' is not defined

In [ ]:
print(product_recommendation_content.columns)
